Connect to google drive

In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


Code to upload files to google drive

In [0]:
# upload file
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def upload(name):
  uploaded = drive.CreateFile({'title': name})
  uploaded.SetContentFile(name)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

Read input video

In [0]:
FOLDERPATH = '/content/drive/My Drive/636/data/raw_video_2/'

Video to frame

In [3]:
# cd "/content/drive/My Drive/636/data/raw_video_2"
cd "/content/drive/My Drive/636/data/clips"

/content/drive/My Drive/636/data/clips


In [7]:
ls

frames1/   frames20/  frames31/  frames42/  frames53/  frames64/  frames75/
frames10/  frames21/  frames32/  frames43/  frames54/  frames65/  frames76/
frames11/  frames22/  frames33/  frames44/  frames55/  frames66/  frames77/
frames12/  frames23/  frames34/  frames45/  frames56/  frames67/  frames78/
frames13/  frames24/  frames35/  frames46/  frames57/  frames68/  frames79/
frames14/  frames25/  frames36/  frames47/  frames58/  frames69/  frames8/
frames15/  frames26/  frames37/  frames48/  frames59/  frames7/   frames80/
frames16/  frames27/  frames38/  frames49/  frames6/   frames70/  frames9/
frames17/  frames28/  frames39/  frames5/   frames60/  frames71/
frames18/  frames29/  frames4/   frames50/  frames61/  frames72/
frames19/  frames3/   frames40/  frames51/  frames62/  frames73/
frames2/   frames30/  frames41/  frames52/  frames63/  frames74/


In [0]:
%%shell
# # video resize
# for i in {1..85}
# {
#     ffmpeg -i $i.mp4 -vf scale=320:-1 0$i.mp4
# }
# frame resize
for i in {2..80}
{
  cd ../frames$i
  for i in *.png
  {
    ffmpeg -i $i -vf scale=320:-1 r$i
  }  
}

In [0]:
# %%shell
# for i in {1..85}
# {
#     mkdir video$i
#     ffmpeg -i 0$i.mp4 video$i/%d.png
# }

In [0]:
%%shell
for i in {31..32}
{
    mkdir video$i
    ffmpeg -i 0$i.mp4 video$i/%d.png
}

Use openpose to generate frames (may take a long time)

In [0]:
cd ~
cd ../content

/content


In [12]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Selecting previously unselected package libgflags2.2.
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

Generate facial landmarks input from frames

In [0]:
# detect poses of images
%%shell
!cd openpose
for i in {5..38}
{
   ./build/examples/openpose/openpose.bin --image_dir ../drive/My\ Drive/636/data/raw_frame/video$i/ --write_json ../drive/My\ Drive/636/data/raw_frame/video$i/ --display 0 --render_pose 0 --face
}

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
In order to save the rendered frames (`--write_images` or `--write_video`), you cannot disable `--render_pose`.
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
^C


In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path

In [0]:
N_FEATURES = 210
folderpath = '/content/drive/My Drive/636/raw_frame/'
COL_NAMES = ['f_' + str(i) for i in range(N_FEATURES)]

In [0]:
# collect the json output of openpose and combine them into dataframe
def json_to_df(folderpath):
  row = 0
  matrix = []
  for name in os.listdir(folderpath):
    if (name[-4:] == 'json'):
      filepath = os.path.join(folderpath, name)
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            matrix.append([])
            for j in range(N_FEATURES):
              matrix[row].append(float(data[0]['face_keypoints_2d'][j]))
          else:
            cur_row = [0 for i in range(210)]
            matrix.append(cur_row)
          row += 1
  df = pd.DataFrame(matrix, columns = COL_NAMES)
  return df

In [0]:
# dataframe to csv and upload to google drive (not necessary)
def upload_csv(df, i):
  name = 'sample_' + str(i) + '.csv'
  df.to_csv(name, index=False)
  upload(name)

Reshape Data into Segments and Prepare for Keras

In [0]:
# slice every 10 frame as a segment until the last frame reaches the end
def get_segments_and_timestamps(df, fps, STEP):
  slices = []
  ts = []
  size = df.shape[0]
  i = STEP
  if size >= STEP:
    while i <= size:
      if i >= STEP:
        slices.append(df.values[df.index[i - STEP : i]])
        ts.append(i/fps)
      i += 1
  else:
    slice_ = df.values
    fillzero = np.zeros((STEP - size, N_FEATURES))
    slice_ = np.concatenate((slice_, fillzero), axis=0)
    slices.append(slice_)
  slices_arr = np.asarray(slices, dtype= np.float32)
  ts_arr = np.asarray(ts, dtype= np.float32)
  return slices_arr, ts_arr